**# House Price Prediction**

## Data Preprocessing and feature engineering

**Author** - `Prasad Parjane`

**Source**-(https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data?select=train.csv)

**Objective** - To predict the sales price of each house using the feature engineering and advanced regression techniques like random forest and gradient boosting



In [ ]:
# import required python libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**STEP 1 : LOAD THE DATASET AMD INSPECTION OF DATASET**

Importing the required python libraries. Thereafter loading the dataset and inspecting the dataset by printing the first 5 rows of the dataset

DATASET : 
1. test.csv
2. train.csv

In [ ]:
df = pd.read_csv(r"C:\Users\Asus Vivobook 15\Desktop\CSI\Week5_Assignment\Data_Files\train.csv")
print(f"Shape of the dataset: {df.shape}")

print("\nColumns in the dataset:")
df.info()

print("\nFirst few rows of the dataset:")
print(df.head())

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (35122919.py, line 1)